In [0]:
# Explain query
spark.sql("SELECT * FROM ecommercetest.silver.silverevents WHERE event_type='purchase'").explain(True)


== Parsed Logical Plan ==
'Project [*]
+- 'Filter ('event_type = purchase)
   +- 'UnresolvedRelation [ecommercetest, silver, silverevents], [], false

== Analyzed Logical Plan ==
event_time: timestamp, event_type: string, product_id: int, category_id: bigint, category_code: string, brand: string, price: double, user_id: int, user_session: string, ingestion_ts: timestamp, event_date: date, price_tier: string
Project [event_time#13198, event_type#13199, product_id#13200, category_id#13201L, category_code#13202, brand#13203, price#13204, user_id#13205, user_session#13206, ingestion_ts#13207, event_date#13208, price_tier#13209]
+- Filter (event_type#13199 = purchase)
   +- SubqueryAlias ecommercetest.silver.silverevents
      +- Relation ecommercetest.silver.silverevents[event_time#13198,event_type#13199,product_id#13200,category_id#13201L,category_code#13202,brand#13203,price#13204,user_id#13205,user_session#13206,ingestion_ts#13207,event_date#13208,price_tier#13209] parquet

== Optimized

In [0]:
# Partitioned table
spark.sql("""
  CREATE TABLE silver_events_part
  USING DELTA
  PARTITIONED BY (event_date, event_type)
  AS SELECT * FROM ecommercetest.silver.silverevents
""")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
# Optimize
spark.sql("OPTIMIZE silver_events_part ZORDER BY (user_id, product_id)")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

In [0]:
# Benchmark
import time
start = time.time()
spark.sql("SELECT * FROM silver_events_part WHERE user_id=12345").count()
print(f"Time: {time.time()-start:.2f}s")

Time: 1.17s
